In [1]:
# default_exp modeling.dataset_utils

In [14]:
#export
from dfdc.core.core import read_metadata
from fastai.vision import *

In [3]:
data_path = Path("/home/ubuntu/data/dfdc"); print(data_path)
video_path = data_path/'dfdc_train'; print(video_path)

/home/ubuntu/data/dfdc
/home/ubuntu/data/dfdc/dfdc_train


In [4]:
#export
def sort_dict(d): return {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}

In [17]:
#export
def flatten_df(df):
    res = []
    for _, row in df.iterrows():
        video_fname = row['video_fname']
        frame_fnames = row['frame_fnames']
        label = row['label']
        for frame_fname in frame_fnames:
            res.append({'video_fname': video_fname, 'frame_fname': frame_fname, 'label':label})
    return pd.DataFrame(res)

In [18]:
#export
def convert_cropped_files_todf(cropped_files:List[PathOrStr]):
    "sort all frames by no and create dataframe for given videos"
    d = defaultdict(dict)
    for o in cropped_files:
        fname = o.parent.name
        frame_fname = str(o)
        frame_no = int(o.name.split("_")[1])
        d[fname][frame_fname] = frame_no

    res = []
    for video_fname, crop_fnames in d.items():
        frame_fnames = list(sort_dict(d[video_fname]).keys())
        res.append({"video_fname":video_fname, "frame_fnames":frame_fnames})
    return pd.DataFrame(res)


def create_frame_label_df(data_path, video_path, part_no):
    cropped_path = data_path/f'dfdc_cropped_faces/dfdc_train_part_{part_no}'
    cropped_files = get_files(cropped_path, extensions=['.jpg'], recurse=True)
    df = convert_cropped_files_todf(cropped_files)
    metadf = read_metadata(video_path/f'dfdc_train_part_{part_no}/metadata.json')
    metadf['video_fname'] = metadf.fname.apply(lambda o: Path(o).stem)
    metadf['label'] = metadf['label'].map({"REAL":0, "FAKE":1})
    merged_df = df.merge(metadf[['video_fname', 'label']], on="video_fname", how="left")
    assert not sum(merged_df.isna().any())
    return merged_df

In [19]:
train_df = create_frame_label_df(data_path, video_path, 49)
valid_df = create_frame_label_df(data_path, video_path, 48)

In [20]:
train_df.shape, valid_df.shape

((3121, 3), (2450, 3))

In [21]:
train_df.label.mean(), valid_df.label.mean()

(0.8349887856456264, 0.8277551020408164)

In [22]:
train_df.head(2)

,video_fname,frame_fnames,label
0,gdnbcgtfay,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1
1,klavqvgpye,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1


In [23]:
valid_df.head(2)

,video_fname,frame_fnames,label
0,bnfhgczmrn,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1
1,ojxeusdtmm,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1


In [25]:
flat_train_df = flatten_df(train_df)

### export

In [26]:
from nbdev.export import notebook2script

In [27]:
notebook2script()

Converted 001 - extract_faces.ipynb.
Converted 002 - face_detection_retinaface.ipynb.
Converted 003 - save_face_crops.ipynb.
Converted 004 - tl_baseline.ipynb.
Converted 00_core.ipynb.
Converted 01_video_core.ipynb.
Converted 10_bbox_utils.ipynb.
Converted 11_retinaface_detection.ipynb.
Converted 12_generate_face_detections.ipynb.
Converted 13_save_cropped_faces.ipynb.
Converted 20_datasets.ipynb.
Converted 21_single_frame_model.ipynb.
Converted index.ipynb.
